In [ ]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import absl.logging
import tensorflow as tf


def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
X = X.reshape(90, 1001, 62, 1)
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

hc_indices = list(range(0, 51))
pg_indices = list(range(51, 90))

fold_val_index_list = []
fold_accuracy = []

input_shape = (1001, 62, 1)

for i in range(len(dataset)): # loso
    test_index = [i]

    hc_val = random.sample([idx for idx in hc_indices if idx not in test_index], 5)
    pg_val = random.sample([idx for idx in pg_indices if idx not in test_index], 5)

    validation_indices = hc_val + pg_val

    training_indices = [idx for idx in range(90) if idx not in test_index + validation_indices]

    fold_val_index_list.append({
        'fold' : i, 
        'test_index' : test_index, 
        'validation_indices' : validation_indices
    })
    
    X_train = X[training_indices, :,  :]
    y_train = y[training_indices, :]
    X_test = X[test_index, :,  :]
    y_test = y[test_index, :]
    X_val = X[validation_indices, :,  :]
    y_val = y[validation_indices, :]

    # print(f"X_train_shape : {X_train.shape}")
    # print(f"X_test_shape : {X_test.shape}")
    # print(f"y_train_shape : {y_train.shape}")
    # print(f"y_test_shape : {y_test.shape}")
    # print(f"X_validation_shape : {X_val.shape}")
    # print(f"y_validation_shape : {y_val.shape}")
    
    model_input = layers.Input(shape = input_shape)
    br1 = layers.Conv2D(16, (50, 1), padding = 'same')(model_input) # temporal filter
    br1 = layers.Activation(mish)(br1) # result1_matrix, shape=(1001, 62, 32)
    # print(br1.shape)

    input_shape_tr = layers.Permute((2, 1, 3))(model_input) # (62, 1001, 1)
    br2 = layers.Conv2D(16, (62, 1), padding = 'same')(input_shape_tr)
    br2 = layers.Activation(mish)(br2) # result2_matrix, shape=(62, 1001, 32)

    # br1, br2 matrix multiply
    br1 = layers.Permute((3, 1, 2))(br1) # (32, 1001, 62)
    br2 = layers.Permute((3, 1, 2))(br2) # (32, 62, 1001)
    br_mul = layers.Lambda(lambda x: tf.matmul(x[0], x[1]))([br1, br2])
    print(br_mul.shape)  # (32, 62, 62)

    # branch matrix multiply convolution
    br_mul = layers.Permute((2, 3, 1))(br_mul) # (62, 62, 32) <- 풀링하기 위함
    br_mul = layers.Conv2D(16, (3, 3), padding = 'same')(br_mul)
    br_mul = layers.Activation(mish)(br_mul)
    br_mul = layers.GlobalAveragePooling2D()(br_mul)
    end = layers.Flatten()(br_mul)

    prediction = layers.Dense(2, activation = 'softmax')(end)
    model = keras.Model(inputs = model_input, outputs = prediction)
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs=1000, batch_size=1, verbose=2)
    predictions = model.predict(X_test)
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)
    fold_accuracy.append(acc)

    # model.summary()
    
    print(f"fold {i} test accuracy : {acc}\n")

    subject_test_acc.append(acc)
scipy.io.savemat('fold_val_indices.mat', {'fold_val_index_list': fold_val_index_list})
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")

print(f"hc : {acc[:51].count(1)}")
print(f"ptsd : {acc[51:].count(1)}")

(None, 16, 1001, 1001)
Epoch 1/100
79/79 - 19s - 244ms/step - accuracy: 0.6076 - loss: 0.9982 - val_accuracy: 0.7000 - val_loss: 0.5942
Epoch 2/100
79/79 - 19s - 235ms/step - accuracy: 0.7215 - loss: 0.7048 - val_accuracy: 0.5000 - val_loss: 0.9781
Epoch 3/100
79/79 - 19s - 236ms/step - accuracy: 0.7089 - loss: 0.6184 - val_accuracy: 0.5000 - val_loss: 0.8386
Epoch 4/100
79/79 - 19s - 241ms/step - accuracy: 0.6962 - loss: 0.7703 - val_accuracy: 0.9000 - val_loss: 0.4885
Epoch 5/100
79/79 - 19s - 241ms/step - accuracy: 0.7342 - loss: 0.6029 - val_accuracy: 0.9000 - val_loss: 0.3876
Epoch 6/100
79/79 - 19s - 240ms/step - accuracy: 0.7468 - loss: 0.6541 - val_accuracy: 0.6000 - val_loss: 0.6266
Epoch 7/100
79/79 - 19s - 244ms/step - accuracy: 0.7089 - loss: 0.5705 - val_accuracy: 0.5000 - val_loss: 1.2094
Epoch 8/100
79/79 - 19s - 240ms/step - accuracy: 0.6835 - loss: 0.6338 - val_accuracy: 0.9000 - val_loss: 0.4185
Epoch 9/100
79/79 - 19s - 240ms/step - accuracy: 0.7215 - loss: 0.5277 - 

In [5]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 1001, 62, 1)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_8 (Permute)           │ (None, 62, 1001, 1)       │               0 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 1001, 62, 16)      │             816 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_6 (Conv2D)             │ (None, 62, 1001, 16)      │           1,008 │ permute_8[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_4 (Activation)     │ (None, 1001, 62, 16)      │               0 │ conv2d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_5 (Activation)     │ (None, 62, 1001, 16)      │               0 │ conv2d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_9 (Permute)           │ (None, 16, 1001, 62)      │               0 │ activation_4[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_10 (Permute)          │ (None, 16, 62, 1001)      │               0 │ activation_5[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lambda_2 (Lambda)             │ (None, 16, 1001, 1001)    │               0 │ permute_9[0][0],           │
│                               │                           │                 │ permute_10[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_11 (Permute)          │ (None, 1001, 1001, 16)    │               0 │ lambda_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_7 (Conv2D)             │ (None, 1001, 1001, 16)    │           2,320 │ permute_11[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_6 (Activation)     │ (None, 1001, 1001, 16)    │               0 │ conv2d_7[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling2d_1    │ (None, 16)                │               0 │ activation_6[0][0]         │
│ (GlobalAveragePooling2D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten_2 (Flatten)           │ (None, 16)                │               0 │ global_average_pooling2d_… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 2)                 │              34 │ flatten_2[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 12,536 (48.97 KB)

 Trainable params: 4,178 (16.32 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,358 (32.65 KB)